In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime
import boto3
import psycopg2
import configparser

#### Iniciación de Variables

In [2]:
cantidad_clientes = np.random.randint(500, 1000) #cantidad de clientes a crear
rdsIdentifier = 'db_proyecto' #nombre de la instancia
fake = Faker() #inicialización para creación de data random

#### Cargamos archivo de configuraciones

In [3]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

### Creamos Instancia de RDS 

In [4]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')

#### Verificamos Instancias de RDS disponibles

In [5]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db-v1']


#### Creación de Servicio RDS

In [6]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

ClientError: An error occurred (InvalidParameterValue) when calling the CreateDBInstance operation: The parameter DBInstanceIdentifier is not a valid identifier. Identifiers must begin with a letter; must contain only ASCII letters, digits, and hyphens; and must not end with a hyphen or contain two consecutive hyphens.

##### Recordemos Esperar unos minutos para consultar la informaicón de la instancia.

##### Obtenemos URL del Host

In [ ]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

banco-db-v1.cadi2n4znidj.us-east-1.rds.amazonaws.com


##### Conexión a Base de Datos desde Python

In [ ]:
from database_DDL import DDL
try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(DDL)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


### Lectura data en S3

In [16]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name


import io
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)
dfs = {}
for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        df_temp = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        dfs[remoteFile] = df_temp
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

dfs

{'countries.xlsx':      country_id ISO_code Continent         Country
 0             1       AF      Asia     Afghanistan
 1             2       AL    Europe         Albania
 2             3       DZ    Africa         Algeria
 3             4       AD    Europe         Andorra
 4             5       AO    Africa          Angola
 ..          ...      ...       ...             ...
 191         192       VN      Asia         Vietnam
 192         193       EH    Africa  Western Sahara
 193         194       YE      Asia           Yemen
 194         195       ZM    Africa          Zambia
 195         196       ZW    Africa        Zimbabwe
 
 [196 rows x 4 columns],
 'purchases data.xlsx':       purchase_id                              user_id         country  \
 0        26854356  google-oauth2|102267249947183414840     Switzerland   
 1        26854413  google-oauth2|102267249947183414840     Switzerland   
 2        26857272       auth0|60e482d0fbcb4800692cdd97         Germany   
 3      

In [18]:
countries = dfs[remoteFileList[0]]
df = dfs[remoteFileList[1]]

# checkpoint> por hacer:
# -- tranformar data:
#     crear todas las dimensiones (datedim)
#     hacer join con country para sacar country id
# Preguntas a resolver:
    # paises con mayor cantidad de ventas
    # dia y hora con mas ventas
    # analisis de discounts> cual se usa mas en que pais
    # el paquete que mas se utilzia y el de mayores ingresos
    # top package por continente
    # cuantos paquetes por usuario se compran 
    # proporicion de compras con descuento y sin descuento por pais 

# df_factTable['year'] = pd.DatetimeIndex(df_factTable['fecha_hora']).year
# df_factTable['month'] = pd.DatetimeIndex(df_factTable['fecha_hora']).month
# df_factTable['day'] = pd.DatetimeIndex(df_factTable['fecha_hora']).day
# df_factTable['hour'] = pd.DatetimeIndex(df_factTable['fecha_hora']).hour
# df_factTable['minute'] = pd.DatetimeIndex(df_factTable['fecha_hora']).minute
# df_factTable['id_fecha'] = df_factTable['year'].astype(str) + df_factTable['month'].astype(str)
# df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['day'].astype(str)
# df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['hour'].astype(str)
# df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['minute'].astype(str)
# factTable_transacciones = df_factTable.drop(['year', 'month', 'day', 'hour', 'minute', 'fecha_hora'], axis=1)
# factTable_transacciones.head()



df

,purchase_id,user_id,country,discountCode,packageType,date
0,26854356,google-oauth2|102267249947183414840,Switzerland,CANCANCAN1,hobbyist,2021-07-06 14:33:08+00:00
1,26854413,google-oauth2|102267249947183414840,Switzerland,CANCANCAN1,trader,2021-07-06 14:35:07+00:00
2,26857272,auth0|60e482d0fbcb4800692cdd97,Germany,NaN,hobbyist,2021-07-06 16:21:26+00:00
3,26860617,auth0|60e4a19f7e80be0071e5db55,USA,NaN,hobbyist,2021-07-06 18:33:31+00:00
4,26861131,auth0|60842ddc5a12e7006a39238e,Germany,NaN,trader,2021-07-06 18:48:50+00:00
...,...,...,...,...,...,...
6072,24136785,auth0|605c97b01b38260069b12505,USA,NaN,trader,2021-03-26 01:49:10+00:00
6073,24144096,auth0|5ff48f08fed6a900752fa6c6,USA,NaN,trader,2021-03-26 09:55:12+00:00
6074,24146741,auth0|605db8536eb2390069abe62a,United Kingdom,FRIEND-TMK041UH,pro,2021-03-26 11:58:18+00:00
6075,24146914,auth0|605dc59b0497600068023158,Switzerland,NaN,hobbyist,2021-03-26 12:05:41+00:00


##### Insertamos Datos en la Tabla de Transacciones

In [11]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [8]:
packages = [

'hobbyist',
'pro',
'tracker',
'trader'
]
a = 10
package_data_real = []
for package in packages:
    package_data = {}

    package_data['name_package'] = package
    package_data['price'] = a
    a += 10
    package_data_real.append(package_data)

package_data_real

[{'name_package': 'hobbyist', 'price': 10},
 {'name_package': 'pro', 'price': 20},
 {'name_package': 'tracker', 'price': 30},
 {'name_package': 'trader', 'price': 40}]

In [12]:
data_tipo_transaciones = [
     {'id_tipo_transac': 85095, 'tipo_transaccion': 'Depóito'}, 
     {'id_tipo_transac': 85098, 'tipo_transaccion': 'Retiro'},
     {'id_tipo_transac': 85194, 'tipo_transaccion': 'Transferencia'},
     {'id_tipo_transac': 85133, 'tipo_transaccion': 'Pago Prestamo'}
]

#insertamos data en tabla tipo_transaccione 
insertDataToSQL(data_tipo_transaciones, 'tipo_transacciones')

Se han insertado 4 nuevos registros.


#### Insertamos Data para Divisas

In [13]:
data_divisas = [
     {'id_divisa': 103589, 'nombre_divisa': 'Quetzales', 'simbolo': 'Q'}, 
     {'id_divisa': 114589, 'nombre_divisa': 'Dolares', 'simbolo': '$'},
     {'id_divisa': 125487, 'nombre_divisa': 'Euros', 'simbolo': '€'}
]

insertDataToSQL(data_divisas, 'divisas')

Se han insertado 3 nuevos registros.


##### Insertamos Data para tipo_documentos

In [14]:
data_tipo_documentos = [
     {'id_tipo_documento': 103589, 'nombre_tipo_documento': 'Recibo'}, 
     {'id_tipo_documento': 114589, 'nombre_tipo_documento': 'Cheque'},
     {'id_tipo_documento': 125487, 'nombre_tipo_documento': 'Contrato 101' }
]

insertDataToSQL(data_tipo_documentos, 'tipo_documentos')


Se han insertado 3 nuevos registros.


##### Insertamos Data para Generos

In [15]:
data_generos = [
     {'id_genero': 2002, 'genero': 'Maculino'}, 
     {'id_genero': 2003, 'genero': 'Femenino'}
]

insertDataToSQL(data_generos, 'generos')

Se han insertado 2 nuevos registros.


##### Insertamos Data para Sectores

In [16]:
data_sectores = [
    {'id_sector': 58974, 'nombre': 'Centro', 'latitud': 90.5689, 'longitud': 91.5687},
    {'id_sector': 58978, 'nombre': 'Centro Sur', 'latitud': 90.5981, 'longitud': 91.56878},
    {'id_sector': 58998, 'nombre': 'Centro Norte', 'latitud': 90.59875, 'longitud': 91.59897},
    {'id_sector': 58784, 'nombre': 'Sur Este', 'latitud': 90.78785, 'longitud': 91.4578},
    {'id_sector': 78786, 'nombre': 'Norte Este', 'latitud': 90.5689, 'longitud': 91.5687},
    {'id_sector': 56565, 'nombre': 'Sur Oeste', 'latitud': 90.4412, 'longitud': 91.9897},
    {'id_sector': 58889, 'nombre': 'Norte Oeste', 'latitud': 90.78745, 'longitud': 91.78897}
]
insertDataToSQL(data_sectores, 'sectores')

Se han insertado 7 nuevos registros.


##### Insertamos Data para Direcciones

In [17]:
data_direcciones = []

for index, direccion in enumerate(range(cantidad_clientes)):
    sector_random = random.sample(data_sectores,1)
    nueva_direccion = {
          'id_direccion': index,
          'no_casa': str(np.random.randint(1, 99)) + '-' + str(np.random.randint(1, 99)),
          'avenida': np.random.randint(1, 45),
          'calle':np.random.randint(1, 50),
          'zona':np.random.randint(1, 21),
          'id_sector' : sector_random[0]['id_sector']
    }

    data_direcciones.append(nueva_direccion)
insertDataToSQL(data_direcciones, 'direcciones')

Se han insertado 966 nuevos registros.


##### Insertamos Data para Clientes

In [18]:
data_clientes = []
data_direcciones_copy = data_direcciones.copy() #copia por valor.

for cliente in range(cantidad_clientes):
    clientProfile = fake.profile()
    direccion_random = random.sample(data_direcciones_copy, 1)
    data_direcciones_copy.remove(direccion_random[0])
    nuevo_cliente = {
        'id_cliente': random.randint(10000000, 99999999),
        'nombre': clientProfile['name'].split(' ')[0],
        'apellido': clientProfile['name'].split(' ')[1],
        'correo': clientProfile['mail'],
        'fecha_nacimiento': clientProfile['birthdate'],
        'telefono': clientProfile['ssn'],
        'id_direccion': direccion_random[0]['id_direccion'],
        'id_genero': data_generos[0]['id_genero'] if(clientProfile['sex'] == 'M') else data_generos[1]['id_genero'] 
    }

    data_clientes.append(nuevo_cliente)

insertDataToSQL(data_clientes, 'clientes')

Se han insertado 966 nuevos registros.


##### Insertamos Data de Cargos

In [19]:
data_cargos = [
    {'id_cargo': 101, 'titulo_cargo': 'Cajero - Receptor'},
    {'id_cargo': 202, 'titulo_cargo': 'Gerente de Sucursal'},
    {'id_cargo': 303, 'titulo_cargo': 'Supervisor de Sector'}
]
insertDataToSQL(data_cargos, 'cargos')

Se han insertado 3 nuevos registros.


##### Insertamos Data de Colaboradores

In [20]:
cantidad_colaboradores = np.random.randint(50, 150)
data_colaboradores = []

for colab in range(cantidad_colaboradores):
    colaboradorProfile = fake.profile()
    nuevo_colaborador = {
        'id_colaborador': random.randint(10000, 99999),
        'nombre': colaboradorProfile['name'].split(' ')[0],
        'apellido': colaboradorProfile['name'].split(' ')[1], 
        'id_cargo': random.choices(data_cargos, weights=(0.6, 0.3, 0.1),  k=1)[0]['id_cargo']
    }

    data_colaboradores.append(nuevo_colaborador)
insertDataToSQL(data_colaboradores, 'colaboradores')

Se han insertado 106 nuevos registros.


In [32]:
managers = [colaborator for colaborator in data_colaboradores if colaborator['id_cargo'] == 202]
cantidad_sucursales =  len(managers)
managers_copy = managers.copy()
data_sucursales = []
pd.DataFrame(managers_copy)

,id_colaborador,nombre,apellido,id_cargo
0,20939,Jacqueline,Adams,202
1,50492,Bruce,Anderson,202
2,44142,Albert,Ochoa,202
3,93924,Stephanie,Contreras,202
4,93471,Victor,Espinoza,202
5,57766,Robert,Gonzales,202
6,66144,Darrell,Wright,202
7,11926,Miss,Rebecca,202
8,93933,William,Miller,202
9,73342,Jean,Williams,202


##### Insertamos Data de Sucursales

In [33]:
managers = [colaborator for colaborator in data_colaboradores if colaborator['id_cargo'] == 202]
cantidad_sucursales =  len(managers)
managers_copy = managers.copy()
data_sucursales = []

for sucursal in range(cantidad_sucursales):
    #seleccionamos gerente para sucursal
    manager = random.sample(managers_copy, 1)[0]
    managers_copy.remove(manager)

    #seleccionamos sector.
    sector_random = random.sample(data_sectores,1)[0]['id_sector']
    
    #nombre de la sucursal
    sucursal_profile = fake.profile()
    nombre_sucursal = sucursal_profile['residence'].split('\n')[0]
    nombre_sucursal = ''.join([i for i in nombre_sucursal if not i.isdigit()]).strip()

    nueva_sucursal = {
        'id_sucursal': np.random.randint(180000, 190000),
        'nombre': nombre_sucursal,
        'id_sector': sector_random,
        'id_gerente': manager['id_colaborador']
    }

    data_sucursales.append(nueva_sucursal)

insertDataToSQL(data_sucursales, 'sucursales')

Se han insertado 36 nuevos registros.


##### Insertamos Datos de tipo_cuentas

In [22]:
data_tipoCuentas = [
    {'id_tipo_cuenta': 90025, 'tipo_cuenta': 'Monetaria Quetzales'},
    {'id_tipo_cuenta': 90026, 'tipo_cuenta': 'Monetaria Dolares'},
    {'id_tipo_cuenta': 90036, 'tipo_cuenta': 'Ahorro Quetzales'},
    {'id_tipo_cuenta': 90078, 'tipo_cuenta': 'Ahorro Dolares'},
    {'id_tipo_cuenta': 90044, 'tipo_cuenta': 'Cheques Quetzales'},
    {'id_tipo_cuenta': 90019, 'tipo_cuenta': 'Cheques Dolares'}
]
insertDataToSQL(data_tipoCuentas, 'tipos_cuentas')

Se han insertado 6 nuevos registros.


##### Insertamos Data para Cuentas

In [23]:
data_cuentas = []
df_data_clientes = pd.DataFrame.from_records(data_clientes)

for idnex, df_row in df_data_clientes.iterrows():
     nombre_cuenta = df_row['nombre'] + ' ' + df_row['apellido']
     nueva_cuenta = {
          'id_cuenta' : np.random.randint(1000000, 9999999),
          'nombre_cuenta' : nombre_cuenta,
          'id_tipo_cuenta': random.sample(data_tipoCuentas, 1)[0]['id_tipo_cuenta'],
          'balance': round(random.uniform(100, 1000), 2)
     }

     data_cuentas.append(nueva_cuenta)
insertDataToSQL(data_cuentas, 'cuentas')

Se han insertado 966 nuevos registros.


##### Insertamos Data de Transacciones

In [34]:
cantidad_transacciones = np.random.randint(1000, 2000)
data_transacciones = []

for transac in range(cantidad_transacciones):
    nueva_transaccion = {
            'id_transaccion': transac,
            'fecha_hora': fake.date_time_this_year(),
            'monto': round(random.uniform(1000, 10000), 2),
            
            'id_tipo_transaccion': random.sample(data_tipo_transaciones, 1)[0]['id_tipo_transac'],
            'id_divisa':  random.sample(data_divisas, 1)[0]['id_divisa'],
            'id_cliente': random.sample(data_clientes, 1)[0]['id_cliente'],
            'id_sucursal': random.sample(data_sucursales, 1)[0]['id_sucursal'],
            'id_colaborador': random.sample(data_colaboradores, 1)[0]['id_colaborador'],
            'id_documento_respaldo': random.sample(data_tipo_documentos, 1)[0]['id_tipo_documento'],
            'id_cuenta_origen' : random.sample(data_cuentas, 1)[0]['id_cuenta'],
            'id_cuenta_destino':random.sample(data_cuentas, 1)[0]['id_cuenta']
        }
    
    data_transacciones.append(nueva_transaccion)
   
insertDataToSQL(data_transacciones, 'transacciones')

Se han insertado 258 nuevos registros.


In [42]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    

query = 'select * from generos'
df_generos = pd.read_sql(query, postgres_driver)

query = 'select * from clientes'
df_clientes = pd.read_sql(query, postgres_driver)

query = 'select * from direcciones'
df_direcciones = pd.read_sql(query, postgres_driver)

query = 'select * from sectores'
df_sectores = pd.read_sql(query, postgres_driver)

In [45]:
dimClientes = df_clientes.merge(df_generos, on='id_genero').merge(df_direcciones, on='id_direccion').merge(df_sectores, on='id_sector', suffixes=('_cliente', '_sector'))
dimClientes = dimClientes.drop(columns=['id_genero', 'id_sector', 'id_direccion' ])
dimClientes

,id_cliente,nombre_cliente,apellido,correo,fecha_nacimiento,telefono,genero,no_casa,avenida,calle,zona,nombre_sector,latitud,longitud
0,84301266,Jasmine,Mills,karinasharp@yahoo.com,1926-11-24,137-65-2220,Femenino,15-16,18,46,18,Centro Norte,90.59875,91.59897
1,10326661,Tina,Wiley,rebeccawatkins@hotmail.com,1999-01-25,013-38-0790,Femenino,43-43,15,14,9,Centro Norte,90.59875,91.59897
2,29900694,Samantha,Fowler,justin85@hotmail.com,1943-03-15,030-78-9033,Femenino,24-88,1,39,2,Centro Norte,90.59875,91.59897
3,82241218,Ann,Mora,michelleday@hotmail.com,1993-12-22,066-27-4596,Femenino,3-78,28,34,7,Centro Norte,90.59875,91.59897
4,39159197,Dr.,Meghan,smithsamuel@hotmail.com,1945-03-10,886-68-1329,Femenino,33-80,36,46,9,Centro Norte,90.59875,91.59897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,89322645,Michael,Lopez,xmedina@gmail.com,2008-01-29,889-68-9151,Maculino,8-49,17,23,7,Sur Este,90.78785,91.45780
962,12557933,Bill,Jacobs,phillipwalker@hotmail.com,1928-03-11,603-38-4302,Maculino,77-44,38,4,17,Sur Este,90.78785,91.45780
963,30215570,Nathaniel,Adams,wallacepeter@hotmail.com,2017-03-24,199-80-2863,Maculino,85-62,9,5,1,Sur Este,90.78785,91.45780
964,16064836,Thomas,Bradley,michelle23@yahoo.com,1928-01-09,223-75-9819,Maculino,88-83,35,39,19,Sur Este,90.78785,91.45780
